# SVM: Scikit-learn sobre dataset Labeled Faces in the Wild

## O que vamos fazer?
- Descarregar e analisar o dataset Labeled Faces in the Wild (LFW). 
- Pré-processar o dataset.
- Formar um modelo de classificação multiclasse com SVM por validação cruzada. 
- Avaliar a precisão do modelo e representá-la graficamente

O dataset Labeled Faces in the Wild (LFW) é um dos datasets mais usados como exemplo e para avaliar os nossos modelos e algoritmos. É um dataset muito curioso: é uma coleção de fotos de rostos de famosos (em formato JPEG) com 13 233 exemplos, 5 749 classes (rostos de famosos) e 5 828 características (pixeis).

Toda a informação está na sua página de internet oficial: http://vis-www.cs.umass.edu/lfw/

Habitualmente usamos o dataset em 2 tipos de tarefas ou modelos diferentes de classificação:
- Verificação de faces: num conjunto de 2 fotografias, um modelo de classificação binária deve prever se é a mesma pessoa.
- Reconhecimento facial: numa fotografia, o modelo deve classificá-la como pertencendo a um dos famosos identificados (classes).


A sua tarefa será criar um modelo SVM para resolver a tarefa de reconhecimento facial, otimizar os seus hiper-parâmetros por validação cruzada e avaliá-lo.

*Vamos formar um modelo ML que seja capaz de reconhecer rostos de famosos!*

Referências:
- [The Labeled Faces in the Wild face recognition dataset](https://scikit-learn.org/stable/datasets/index.html#labeled-faces-in-the-wild-dataset)
- [sklearn.datasets.fetch_lfw_people](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_lfw_people.html)
- [sklearn.datasets.fetch_lfw_pairs](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_lfw_pairs.html)
- [Faces recognition example using eigenfaces and SVMs](https://scikit-learn.org/stable/auto_examples/applications/plot_face_recognition.html)

In [1]:
# TODO: Importar de todos os módulos necessários para esta célula

from sklearn.datasets import fetch_lfw_people
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from time import time
from scipy.stats import loguniform
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, mean_squared_error


## Carregar o dataset LFW

Representar graficamente alguns dos exemplos e as suas classes ou números associados.

*Nota*: Cuidado com o seu tamanho! Se for um dataset demasiado grande para o seu ambiente, pode reduzir o seu tamanho escolhendo um número de exemplos mais reduzido ao acaso, reordenando aleatoriamente os exemplos para evitar ter classes/faces com um número baixo de exemplos:

### Foi reduzido o Dataset, devido ao facto de ser muito extenso. Apenas faces com no minimo 20 exemplos foram escolhidas e cada imagem foi redimensionada para 30% do tamanho original 

In [22]:
# TODO: Carregar o dataset Digits como arrays X e Y e representar alguns dos exemplos
lfw_people = fetch_lfw_people(min_faces_per_person=20)#, resize=0.4)  # redimensiona as imagens para 30% do tamanho original

n_samples, h, w = lfw_people.images.shape

print(f"Exitem {n_samples} exemplos")
print(f"A imagem tem de tamanho {h} x {w}.")

Exitem 3023 exemplos
A imagem tem de tamanho 62 x 47.


In [23]:
X = lfw_people.data

# the label to predict is the id of the person
y = lfw_people.target
target_names = lfw_people.target_names

print("Tamnho do dataset:", X.shape)
print("Número de classes:", len(target_names))
print("Nome das pessoas:", target_names)

Tamnho do dataset: (3023, 2914)
Número de classes: 62
Nome das pessoas: ['Alejandro Toledo' 'Alvaro Uribe' 'Amelie Mauresmo' 'Andre Agassi'
 'Angelina Jolie' 'Ariel Sharon' 'Arnold Schwarzenegger'
 'Atal Bihari Vajpayee' 'Bill Clinton' 'Carlos Menem' 'Colin Powell'
 'David Beckham' 'Donald Rumsfeld' 'George Robertson' 'George W Bush'
 'Gerhard Schroeder' 'Gloria Macapagal Arroyo' 'Gray Davis'
 'Guillermo Coria' 'Hamid Karzai' 'Hans Blix' 'Hugo Chavez' 'Igor Ivanov'
 'Jack Straw' 'Jacques Chirac' 'Jean Chretien' 'Jennifer Aniston'
 'Jennifer Capriati' 'Jennifer Lopez' 'Jeremy Greenstock' 'Jiang Zemin'
 'John Ashcroft' 'John Negroponte' 'Jose Maria Aznar'
 'Juan Carlos Ferrero' 'Junichiro Koizumi' 'Kofi Annan' 'Laura Bush'
 'Lindsay Davenport' 'Lleyton Hewitt' 'Luiz Inacio Lula da Silva'
 'Mahmoud Abbas' 'Megawati Sukarnoputri' 'Michael Bloomberg' 'Naomi Watts'
 'Nestor Kirchner' 'Paul Bremer' 'Pete Sampras' 'Recep Tayyip Erdogan'
 'Ricardo Lagos' 'Roh Moo-hyun' 'Rudolph Giuliani' 'Sadda

In [4]:
X[:5]

array([[0.31764707, 0.35555556, 0.45228758, ..., 0.42222223, 0.5921569 ,
        0.49019608],
       [0.1385621 , 0.27450982, 0.33464053, ..., 0.23398693, 0.3267974 ,
        0.45359477],
       [0.33594772, 0.21830066, 0.22745098, ..., 0.69542485, 0.32026145,
        0.303268  ],
       [0.5555556 , 0.5542484 , 0.4392157 , ..., 0.45359477, 0.4261438 ,
        0.43790853],
       [0.34901962, 0.38431373, 0.3529412 , ..., 0.8928104 , 0.93202615,
        0.94248366]], dtype=float32)

## Pré-processar os dados

Pré-processar os dados com os métodos Scikit-learn:

- Reordená-los aleatoriamente. 
- Normalizar, se necessário.
- Dividi-los em subsets de formação e testes.

Nesta ocasião, mais uma vez, faremos a validação cruzada por K-fold.

In [24]:
# TODO: Reordenar os dados aleatoriamente

indices = np.arange(len(X))
np.random.shuffle(indices)

X = X[indices]
y = y[indices]
X[:5]

array([[0.23006536, 0.3150327 , 0.4169935 , ..., 0.5620915 , 0.5503268 ,
        0.5673203 ],
       [0.24836601, 0.26797387, 0.2379085 , ..., 0.6379085 , 0.6575164 ,
        0.6653595 ],
       [0.42875817, 0.39346406, 0.35686275, ..., 0.882353  , 0.8366013 ,
        0.41568628],
       [0.26666668, 0.3137255 , 0.29150328, ..., 0.4928105 , 0.07189543,
        0.00261438],
       [0.48758173, 0.5503268 , 0.60915035, ..., 0.0627451 , 0.05620915,
        0.06143792]], dtype=float32)

In [ ]:
# TODO: Normalizar, se necessário

In [25]:
# TODO: Dividi-los em subsets de formação e testes

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

scaler = StandardScaler() # padronizar os dados para que tenham média 0 e desvio padrão 1
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Formar um modelo de classificação por SVM inicial

Para comprovar o funcionamento do nosso classificador SVC antes de o otimizar por validação cruzada, vamos formar um modelo inicial sobre o subset de formação e validá-lo sobre o subset de teste.

Recordar usar a função [decision_function_shape](https://scikit-learn.org/stable/modules/svm.html#multi-class-classification) para usar o esquema “um contra o resto” (“ovr”). 

Usar os valores por defeito de *C* e *gamma* para não influir sobre a sua regularização:

In [26]:
model = SVC(decision_function_shape='ovr')
t0 = time()
model.fit(X_train, y_train)
print("done in %0.3fs" % (time() - t0))

done in 13.656s


## Avaliar o modelo no subset de teste
Neste caso, não vamos representar a matriz de confusão, pois com 5749 classes seria demasiado grande para o analisar num gráfico:

In [27]:
# TODO: Avaliar o modelo com o seu score () no subset de teste.
t1 = time()
y_test_pred = model.predict(X_test)
print("done in %0.3fs" % (time() - t1))

done in 4.781s


In [28]:
print("Accuracy:", accuracy_score(y_test, y_test_pred))

Accuracy: 0.39814814814814814


### Então concluimos que, provavelmente por usar os valores por defeito de C e gamma, não obteve um bom resultado de accuracy nomeadamente de 0.68.